In [24]:
import os
import json
import random

import nltk
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /Users/hanz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hanz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hanz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/hanz/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [25]:
intents = {
	"intents": [
		{
			"tag": "greeting",
			"patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up", "Hey", "greetings"],
			"responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"]
		},
		{
			"tag": "goodbye",
			"patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day", "bye", "cao", "see ya"],
			"responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"]
		},
		{
			"tag": "programming",
			"patterns": ["What is progamming?", "What is coding?", "Tell me about programming", "Tell me about coding", "What is software development?"],
			"responses": ["Programming, coding or software development, means writing computer code to automate tasks."]
		},
		{
			"tag": "resource",
			"patterns": ["Where can I learn to code?", "Best way to learn to code", "How can I learn programming", "Good programming resources",
								   "Can you recommend good coding resources?"],
			"responses": ["Check out the NeuralNine YouTube channel and The Python Bible series (7 in 1)."]
		},
        {
			"tag": "handsome",
			"patterns": ["who is the most handsome person", "the most handsome people in world", "handsome and cool", "who is more handsome than me", "Goodlooking man"],
			"responses": ["Handashuai!", "hanz is the most handsome people", "It must be handashuai"]
		},
		{
			"tag": "stocks",
			"patterns": ["What are my stocks?", "Which stocks do I own?", "Show my stock portfolio"],
			"responses": ["Here are your stocks!"]
		}
	]
}

with open("intents.json", "w") as f:
    json.dump(intents, f, indent=4)

## model architecture

In [26]:
class ChatbotModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(ChatbotModel, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x
        

## data preparation

In [27]:
class ChatbotAssistant:
    
    def __init__(self, intents_path, function_mappings = None):
        self.model = None
        self.intents_path = intents_path
        
        self.documents = []
        self.vocabulary = []
        self.intents = []
        self.intents_responses = {}
        
        self.function_mappings = function_mappings
        
        self.X = None
        self.y = None
        
    @staticmethod
    def tokenize_and_lemmatize(text):
        lemmatizer = nltk.WordNetLemmatizer()
        
        words = nltk.word_tokenize(text)
        words = [lemmatizer.lemmatize(word.lower()) for word in words]
        
        return words
    
    def bag_of_words(self, words):
        return [1 if word in words else 0 for word in self.vocabulary]
    
    def parse_intents(self):
        lemmatizer = nltk.WordNetLemmatizer()
        
        if os.path.exists(self.intents_path):
            with open(self.intents_path, 'r') as f:
                intents_data = json.load(f)
            
            for intent in intents_data['intents']:
                if intent['tag'] not in self.intents:
                    self.intents.append(intent['tag'])
                    self.intents_responses[intent['tag']] = intent['responses']
                    
                for pattern in intent['patterns']:
                    pattern_words = self.tokenize_and_lemmatize(pattern)
                    self.vocabulary.extend(pattern_words)
                    self.documents.append((pattern_words, intent['tag']))
                
            self.vocabulary = sorted(set(self.vocabulary))
            
    def prepare_data(self):
        bags = []
        indices = []
        
        for document in self.documents:
            words = document[0]
            bag = self.bag_of_words(words)
            
            intent_index = self.intents.index(document[1])
            
            bags.append(bag)
            indices.append(intent_index)
            
        self.X = np.array(bags)
        self.y = np.array(indices)
        
    def train_model(self, batch_size, lr, epochs):
        X_tensor = torch.tensor(self.X, dtype=torch.float32)
        y_tensor = torch.tensor(self.y, dtype=torch.long)

        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.model = ChatbotModel(self.X.shape[1], len(self.intents))

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=lr)

        for epoch in range(epochs):
            running_loss = 0.0

            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            print(f"Epoch: {epoch+1} Loss: {running_loss/ len(loader):.4f} ")

    def save_model(self, model_path, dimensions_path):
        torch.save(self.model.state_dict(), model_path)

        with open(dimensions_path, 'w') as f:
            json.dump({ 'input_size': self.X.shape[1], 'output_size': len(self.intents)}, f)

    def load_model(self, model_path, dimensions_path):
        with open(dimensions_path, 'r') as f:
            dimensions = json.load(f)

            self.model = ChatbotModel(dimensions['input_size'], dimensions['output_size'])
            self.model.load_state_dict(torch.load(model_path, weight_only=True))

    def process_message(self, input_message):
        words = self.tokenize_and_lemmatize(input_message)
        bag = self.bag_of_words(words)

        bag_tensor = torch.tensor([bag], dtype=torch.float32)

        self.model.eval()
        with torch.no_grad():
            predictions = self.model(bag_tensor)

        predicted_class_index = torch.argmax(predictions, dim=1).item()
        predicted_intent = self.intents[predicted_class_index]

        if self.function_mappings:
            if predicted_intent in self.function_mappings:
                self.function_mappings[predicted_intent]()

        if self.intents_responses[predicted_intent]:
            return random.choice(self.intents_responses[predicted_intent])
        else:
            return None
        

In [28]:
os.getcwd()

'/Users/hanz/anaconda3'

In [29]:
def get_stock():
    stocks = ['APPL', 'META', 'GS', 'MSTF']
    
    print(random.sample(stocks, 3))

if __name__ == '__main__':
    assistant = ChatbotAssistant('intents.json', function_mappings = {'stocks': get_stock})
    assistant.parse_intents()
    assistant.prepare_data()
    assistant.train_model(batch_size=8, lr=0.001, epochs=100)
    
    assistant.save_model('chatbot_model.pth', 'dimensions.json')
    
    # assistant = ChatbotAssistant('intents.json', function_mappings = {'stocks': get_stock})
    # assistant.parse_intents()
    # assistant_save_model('chatbot_model.pth', 'dimensions.json')

Epoch: 1 Loss: 1.7861 
Epoch: 2 Loss: 1.7939 
Epoch: 3 Loss: 1.7961 
Epoch: 4 Loss: 1.7801 
Epoch: 5 Loss: 1.7614 
Epoch: 6 Loss: 1.7215 
Epoch: 7 Loss: 1.7376 
Epoch: 8 Loss: 1.7435 
Epoch: 9 Loss: 1.6925 
Epoch: 10 Loss: 1.6977 
Epoch: 11 Loss: 1.6836 
Epoch: 12 Loss: 1.6808 
Epoch: 13 Loss: 1.6557 
Epoch: 14 Loss: 1.6464 
Epoch: 15 Loss: 1.6391 
Epoch: 16 Loss: 1.6072 
Epoch: 17 Loss: 1.5508 
Epoch: 18 Loss: 1.5473 
Epoch: 19 Loss: 1.5220 
Epoch: 20 Loss: 1.3794 
Epoch: 21 Loss: 1.4181 
Epoch: 22 Loss: 1.4101 
Epoch: 23 Loss: 1.2829 
Epoch: 24 Loss: 1.3654 
Epoch: 25 Loss: 1.1697 
Epoch: 26 Loss: 1.3280 
Epoch: 27 Loss: 1.2048 
Epoch: 28 Loss: 1.1039 
Epoch: 29 Loss: 1.0010 
Epoch: 30 Loss: 0.9308 
Epoch: 31 Loss: 0.9140 
Epoch: 32 Loss: 0.9313 
Epoch: 33 Loss: 0.8852 
Epoch: 34 Loss: 0.7027 
Epoch: 35 Loss: 0.6267 
Epoch: 36 Loss: 0.6523 
Epoch: 37 Loss: 0.6316 
Epoch: 38 Loss: 0.5789 
Epoch: 39 Loss: 0.6577 
Epoch: 40 Loss: 0.4632 
Epoch: 41 Loss: 0.4934 
Epoch: 42 Loss: 0.4665 
E

In [ ]:
while True:
    message = input('Enter your message: ')
    if message == 'quit':
        break
        
    print(assistant.process_message(message))

Enter your message: hi
Hello!
Enter your message: show me my stocks
['META', 'APPL', 'GS']
Here are your stocks!
Enter your message: who is the most handsome people in the world
hanz is the most handsome people
Enter your message: thank you
Hi there, how can I help?
Enter your message: introduce me programming
Programming, coding or software development, means writing computer code to automate tasks.
Enter your message: bye
Talk to you later
